# Analysis of sea surface properties
- SST, SSS, SSH, velocity vectors are displayed in animations
- Composites of decadal variability are illustrated 

In [1]:
#Import Packages

import xarray as xr
import matplotlib.pyplot as plt
import numpy as np 
import math
import matplotlib.font_manager
import cmocean as cmo
from tqdm import tqdm
import cartopy.crs as ccrs
import os
from matplotlib import colors
from matplotlib.animation import FuncAnimation 
import cartopy.crs as ccrs
import cartopy.feature as cfeature


#Ignore warnings for now
import warnings
warnings.filterwarnings('ignore')

#Local path to respository (Change to your directory for code to work)
path_rp = '/home/milon.miah/Documents/'

os.chdir(path_rp + 'DV_NAshelf_NEMO/Scripts')
from worldmap import WorldMap
from calc import *
from util import *
os.chdir(path_rp)

#Plot parameters
plt.rcParams['font.size'] = 10.0

## Load Data
### Sea surface

In [2]:
#Load data from vortex directory - Meridonal velocity
vortex_path = '/vortex/clidex/data/NEMO/VIKING20X/hydrography_monthly_upper1000m/'
fname_prefix = '1_VIKING20X.L46-KFS003_1m_'
fname_suffix = '_45W_80W_30N_57N_upper1000m.nc'
ds_mev = xr.open_mfdataset(vortex_path + fname_prefix + '*_vomecrty' + fname_suffix)

#Load data from vortex directory - Zonal velocity
vortex_path = '/vortex/clidex/data/NEMO/VIKING20X/hydrography_monthly_upper1000m/'
fname_prefix = '1_VIKING20X.L46-KFS003_1m_'
fname_suffix = '_45W_80W_30N_57N_upper1000m.nc'
ds_zov = xr.open_mfdataset(vortex_path + fname_prefix + '*_vozocrtx' + fname_suffix)

#Load data from vortex directory - Temperature
vortex_path = '/vortex/clidex/data/NEMO/VIKING20X/hydrography_monthly_upper1000m/'
fname_prefix = '1_VIKING20X.L46-KFS003_1m_'
fname_suffix = '_45W_80W_30N_57N_upper1000m.nc'
ds_temp = xr.open_mfdataset(vortex_path + fname_prefix + '*_votemper' + fname_suffix)

#Load data from vortex directory - Salinity
vortex_path = '/vortex/clidex/data/NEMO/VIKING20X/hydrography_monthly_upper1000m/'
fname_prefix = '1_VIKING20X.L46-KFS003_1m_'
fname_suffix = '_45W_80W_30N_57N_upper1000m.nc'
ds_sal = xr.open_mfdataset(vortex_path + fname_prefix + '*_vosaline' + fname_suffix)

#Load data from vortex directory - SSH
ds_ssh = xr.open_dataset('/mnt/data/SSH/ssh_monthly.nc')
#Change nav_lat and nav_lon into coordinates instead of data variable
ds_ssh = ds_ssh.set_coords(("nav_lat", "nav_lon"))

#Load data from vortex directory - Wind Stress
vortex_path = '/vortex/clidex/data/NEMO/VIKING20X/'
ds_wind_x = xr.open_mfdataset(vortex_path + 'TauX_NA_1m_1958_2019.nc')
ds_wind_y = xr.open_dataset(vortex_path + 'TauY_NA_1m_1958_2019.nc')

In [3]:
# Only look at surface 
ds_mev = ds_mev.sel(depthv = ds_mev['vomecrty'][:,0].depthv.to_numpy())
ds_zov = ds_zov.sel(depthu = ds_zov['vozocrtx'][:,0].depthu.to_numpy())
ds_temp = ds_temp.sel(deptht = ds_temp['votemper'][:,0].deptht.to_numpy())
ds_sal = ds_sal.sel(deptht = ds_sal['vosaline'][:,0].deptht.to_numpy())

# Cut off land
ds_zov = ds_zov.where(ds_zov != 0.0)
ds_mev = ds_mev.where(ds_mev != 0.0)
ds_temp = ds_temp.where(ds_temp != 0.0)
ds_sal = ds_sal.where(ds_sal != 0.0)
ds_ssh = ds_ssh.where(ds_ssh != 0.0)
ds_wind_x = ds_wind_x.where(ds_wind_x != 0.0)
ds_wind_y = ds_wind_y.where(ds_wind_y != 0.0)

### Sections

In [4]:
# Read Data of the three sections
path_sections = '/vortex/clidex/data/NEMO/VIKING20X/sections/section_'

ds_71W =  xr.open_dataset(path_sections + '71W_incl_GS.nc')
ds_GoM = xr.open_dataset(path_sections + 'GoM_entrance_incl_GS.nc')
ds_OL = xr.open_dataset(path_sections + 'Oleander_incl_GS.nc')

#Cut off land mass
ds_71W = ds_71W.where(ds_71W != 0.0)
ds_GoM = ds_GoM.where(ds_GoM != 0.0)
ds_OL = ds_OL.where(ds_OL != 0.0)


In [ ]:
# Take time-mean (optionally)
ds_zov_mean = ds_zov.mean(dim = 'time_counter').load()
ds_mev_mean = ds_mev.mean(dim = 'time_counter').load()
ds_temp_mean = ds_temp.mean(dim = 'time_counter').load()
ds_sal_mean = ds_sal.mean(dim = 'time_counter').load()
ds_wind_x_mean = ds_wind_x.mean(dim = 'time_counter').load()
ds_wind_y_mean = ds_wind_y.mean(dim = 'time_counter').load()

## Time-mean fields

### Velocity vectors

In [ ]:
# Plot map with velocity mean vectors
atl = WorldMap(central_longitude = -60, real_color = True, zoom = [-85, -40, 30, 57])
atl.plot_quiver(ds_zov_mean.nav_lon, ds_zov_mean.nav_lat, ds_zov_mean['vozocrtx'], ds_mev_mean['vomecrty'], skip = 30)

### Wind stress

In [ ]:
#Plot mean wind vectors (Appear very small) 
atl = WorldMap(central_longitude = -60, real_color = True, zoom = [-85, -40, 30, 57])
atl.plot_quiver(ds_wind_x_mean.nav_lon, ds_wind_x_mean.nav_lat, ds_wind_x_mean['sozotaux'], ds_wind_y_mean['sometauy'], skip = 30)

# Animation of velocity vectors and SST, SSS, SSH

### SST

In [ ]:
%matplotlib notebook
#Create Animation for surface velocity and temperature field
#Plot parameters
plt.rcParams['font.size'] = 10.0
#Creating the map object
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

#Adding features to the map
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.LAKES, alpha=0.5)
ax.add_feature(cfeature.RIVERS)
#ax.stock_img()
ax.set_extent([-85, -40, 30, 57])
ax.gridlines(draw_labels = True)

#Define observation period
time_begin = '16-01-1958'
time_end = '16-12-2019'

ds_zov_foc = ds_zov.sel(time_counter = slice(time_begin, time_end))
ds_mev_foc = ds_mev.sel(time_counter = slice(time_begin, time_end))
ds_temp_foc = ds_temp.sel(time_counter = slice(time_begin, time_end))

#Decrease arrow resolution
skip = 10
to_skip = (slice(None, None, skip), slice(None, None, skip))
nav_lon = ds_zov_foc.nav_lon[to_skip]
nav_lat = ds_zov_foc.nav_lat[to_skip]
nav_lon_temp = ds_temp_foc.nav_lon[to_skip]
nav_lat_temp = ds_temp_foc.nav_lat[to_skip]
zov = ds_zov_foc['vozocrtx'][0][to_skip]
mev = ds_mev_foc['vomecrty'][0][to_skip]
temp = ds_temp_foc['votemper'][0][to_skip]

#Add colormesh of temperature plot
vmin = 0
vmax = 30
cmesh = ax.pcolormesh(nav_lon_temp, nav_lat_temp, temp, 
                      transform = ccrs.PlateCarree(), cmap = 'cmo.thermal', vmin = vmin, vmax = vmax)
cbar = fig.colorbar(cmesh, ax = ax, location = 'right', label = 'Temperature [°C]', pad = 0.1)

#Start plotting vector field for time begin
quiv = ax.quiver(nav_lon, nav_lat, zov, mev)

def animate(i):
    
    plt.title(np.datetime_as_string(ds_zov_foc['time_counter'][i].to_numpy(), unit = 'M'))
    cmesh.set_array(ds_temp_foc['votemper'][i][to_skip])
    quiv.set_UVC(ds_zov_foc['vozocrtx'][i][to_skip], ds_mev_foc['vomecrty'][i][to_skip])
    ax.coastlines()

    
# # # anim = FuncAnimation(fig_sctt2, animate, init_func = init, frames = 200, interval = 20, blit = True)
anim = FuncAnimation(fig, animate, frames = len(ds_zov_foc.time_counter.to_numpy()), interval = 5, blit = False)
anim.save(path_rp + 'DV_NAshelf_NEMO/Plots/Maps/surface_temp_vel.gif', writer='imagemagick', fps=5)

### SSS

In [ ]:
%matplotlib notebook
#Create Animation for surface velocity and salinity field
#Plot parameters
plt.rcParams['font.size'] = 10.0
#Creating the map object
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

#Adding features to the map
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.LAKES, alpha=0.5)
ax.add_feature(cfeature.RIVERS)
#ax.stock_img()
ax.set_extent([-85, -40, 30, 57])
ax.gridlines(draw_labels = True)

#Define observation period
time_begin = '16-01-1958'
time_end = '16-12-2019'

ds_zov_foc = ds_zov.sel(time_counter = slice(time_begin, time_end))
ds_mev_foc = ds_mev.sel(time_counter = slice(time_begin, time_end))
ds_sal_foc = ds_sal.sel(time_counter = slice(time_begin, time_end))

#Decrease arrow resolution
skip = 10
to_skip = (slice(None, None, skip), slice(None, None, skip))
nav_lon = ds_zov_foc.nav_lon[to_skip]
nav_lat = ds_zov_foc.nav_lat[to_skip]
nav_lon_sal = ds_sal_foc.nav_lon[to_skip]
nav_lat_sal = ds_sal_foc.nav_lat[to_skip]
zov = ds_zov_foc['vozocrtx'][0][to_skip]
mev = ds_mev_foc['vomecrty'][0][to_skip]
sal = ds_sal_foc['vosaline'][0][to_skip]

#Add colormesh of salinity plot
vmin = 0
vmax = 30
cmesh = ax.pcolormesh(nav_lon_sal, nav_lat_sal, sal, 
                      transform = ccrs.PlateCarree(), vmin = 30, vmax = 37)
cbar = fig.colorbar(cmesh, ax = ax, cmap = 'cmo.haline', location = 'right', label = 'Salinity [g/kg]', pad = 0.1)

#Start plotting vector field for time begin
quiv = ax.quiver(nav_lon, nav_lat, zov, mev)

def animate(i):
    
    plt.title(np.datetime_as_string(ds_zov_foc['time_counter'][i].to_numpy(), unit = 'M'))
    cmesh.set_array(ds_sal_foc['vosaline'][i][to_skip])
    quiv.set_UVC(ds_zov_foc['vozocrtx'][i][to_skip], ds_mev_foc['vomecrty'][i][to_skip])
    ax.coastlines()

    
# # # anim = FuncAnimation(fig_sctt2, animate, init_func = init, frames = 200, interval = 20, blit = True)
anim = FuncAnimation(fig, animate, frames = len(ds_zov_foc.time_counter.to_numpy()), interval = 5, blit = False)
anim.save(path_rp + 'DV_NAshelf_NEMO/Plots/Maps/surface_sal_vel.gif', writer='imagemagick', fps=5)

### SSH

In [ ]:
%matplotlib notebook
#Create Animation for surface velocity and ssh field
#Plot parameters
plt.rcParams['font.size'] = 10.0
#Creating the map object
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

#Adding features to the map
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.LAKES, alpha=0.5)
ax.add_feature(cfeature.RIVERS)
#ax.stock_img()
ax.set_extent([-85, -40, 30, 57])
ax.gridlines(draw_labels = True)

#Define observation period
time_begin = '01-01-2010'
time_end = '31-12-2019'

ds_zov_foc = ds_zov.sel(time_counter = slice(time_begin, time_end))
ds_mev_foc = ds_mev.sel(time_counter = slice(time_begin, time_end))
ds_ssh_foc = ds_ssh.sel(time_counter = slice(time_begin, time_end))

#Decrease arrow resolution
skip = 10
to_skip = (slice(None, None, skip), slice(None, None, skip))
nav_lon = ds_zov_foc.nav_lon[to_skip]
nav_lat = ds_zov_foc.nav_lat[to_skip]
nav_lon_ssh = ds_ssh_foc.nav_lon[to_skip]
nav_lat_ssh = ds_ssh_foc.nav_lat[to_skip]
zov = ds_zov_foc['vozocrtx'][0][to_skip]
mev = ds_mev_foc['vomecrty'][0][to_skip]
ssh = ds_ssh_foc['sossheig'][0][to_skip]

#Add colormesh of ssherature plot
vmin = 0
vmax = 10
cmesh = ax.pcolormesh(nav_lon_ssh, nav_lat_ssh, ssh, 
                      transform = ccrs.PlateCarree(), cmap = 'cmo.balance', norm = colors.CenteredNorm())
cbar = fig.colorbar(cmesh, ax = ax, location = 'right', label = 'Sea Surface Height (Zero mean) [m]', pad = 0.1)

#Start plotting vector field for time begin
quiv = ax.quiver(nav_lon, nav_lat, zov, mev)

def animate(i):
    
    plt.title(np.datetime_as_string(ds_zov_foc['time_counter'][i].to_numpy(), unit = 'M'))
    cmesh.set_array(ds_ssh_foc['sossheig'][i][to_skip])
    quiv.set_UVC(ds_zov_foc['vozocrtx'][i][to_skip], ds_mev_foc['vomecrty'][i][to_skip])
    ax.coastlines()

    
# # # anim = FuncAnimation(fig_sctt2, animate, init_func = init, frames = 200, interval = 20, blit = True)
anim = FuncAnimation(fig, animate, frames = len(ds_ssh_foc.time_counter.to_numpy()), interval = 5, blit = False)
anim.save(path_rp + 'DV_NAshelf_NEMO/Plots/Maps/surface_ssh_vel.gif', writer='imagemagick', fps=5)

### Wind Stress

In [ ]:
%matplotlib notebook
#Create Animation for wind stress
#Plot parameters
plt.rcParams['font.size'] = 10.0
#Creating the map object
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

#Adding features to the map
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.LAKES, alpha=0.5)
ax.add_feature(cfeature.RIVERS)
#ax.stock_img()
ax.set_extent([-85, -40, 30, 57])
ax.gridlines(draw_labels = True)

#Define observation period
time_begin = '01-01-2019'
time_end = '31-12-2019'

ds_wind_x_foc = ds_wind_x.sel(time_counter = slice(time_begin, time_end))
ds_wind_y_foc = ds_wind_y.sel(time_counter = slice(time_begin, time_end))

#Decrease arrow resolution
skip = 10
to_skip = (slice(None, None, skip), slice(None, None, skip))
nav_lon = ds_wind_x_foc.nav_lon[to_skip]
nav_lat = ds_wind_y_foc.nav_lat[to_skip]
wind_x = ds_wind_x_foc['sozotaux'][0][to_skip]
wind_y = ds_wind_y_foc['sometauy'][0][to_skip]


#Start plotting vector field for time begin
quiv = ax.quiver(nav_lon, nav_lat, wind_x, wind_y)

def animate(i):
    
    plt.title(np.datetime_as_string(ds_wind_x_foc['time_counter'][i].to_numpy(), unit = 'M'))
    quiv.set_UVC(ds_wind_x['sozotaux'][i][to_skip], ds_wind_y['sometauy'][i][to_skip])

    
# # # anim = FuncAnimation(fig_sctt2, animate, init_func = init, frames = 200, interval = 20, blit = True)
anim = FuncAnimation(fig, animate, frames = len(ds_wind_x_foc.time_counter.to_numpy()), interval = 5, blit = False)
anim.save(path_rp + 'DV_NAshelf_NEMO/Plots/Maps/surface_wind.gif', writer='imagemagick', fps=5)

# Composites of decadal variability

### Eliminate seasonal cycle 
Time period 1990-2019

In [ ]:
#Time periods of decadal variability
time_periods = ['1960', '1968', '1986', '2012', '2019']

#Calculate anomalies
#Choose data from 1990-2019, group per month, calculate mean and substract from data to obtain anomalies
temp_ref = ds_temp.sel(time_counter = slice('01-01-1990', '31-12-2019')).groupby('time_counter.month').mean(dim = 'time_counter')
temp_anom = ds_temp.groupby('time_counter.month') - temp_ref
sal_ref = ds_sal.sel(time_counter = slice('01-01-1990', '31-12-2019')).groupby('time_counter.month').mean(dim = 'time_counter')
sal_anom = ds_sal.groupby('time_counter.month') - sal_ref
mev_ref = ds_mev.sel(time_counter = slice('01-01-1990', '31-12-2019')).groupby('time_counter.month').mean(dim = 'time_counter')
mev_anom = ds_mev.groupby('time_counter.month') - mev_ref
zov_ref = ds_zov.sel(time_counter = slice('01-01-1990', '31-12-2019')).groupby('time_counter.month').mean(dim = 'time_counter')
zov_anom = ds_zov.groupby('time_counter.month') - zov_ref
ssh_ref = ds_ssh.sel(time_counter = slice('01-01-1990', '31-12-2019')).groupby('time_counter.month').mean(dim = 'time_counter')
ssh_anom = ds_ssh.groupby('time_counter.month') - ssh_ref

temp_periods_anom = {}
sal_periods_anom = {}
mev_periods_anom = {}
zov_periods_anom = {}
ssh_periods_mean = {}
mev_periods = {}
zov_periods = {}
wind_x_periods = {}
wind_y_periods = {}

for j in tqdm(range(len(time_periods) - 1)):
    temp_periods_anom[time_periods[j] + '-' + time_periods[j+1]] = temp_anom.sel(time_counter = slice(time_periods[j], time_periods[j+1])).mean(dim = 'time_counter').compute()
    sal_periods_anom[time_periods[j] + '-' + time_periods[j+1]] = sal_anom.sel(time_counter = slice(time_periods[j], time_periods[j+1])).mean(dim = 'time_counter').compute()
    mev_periods_anom[time_periods[j] + '-' + time_periods[j+1]] = mev_anom.sel(time_counter = slice(time_periods[j], time_periods[j+1])).mean(dim = 'time_counter').compute()
    zov_periods_anom[time_periods[j] + '-' + time_periods[j+1]] = zov_anom.sel(time_counter = slice(time_periods[j], time_periods[j+1])).mean(dim = 'time_counter').compute()
    mev_periods[time_periods[j] + '-' + time_periods[j+1]] = ds_mev.sel(time_counter = slice(time_periods[j], time_periods[j+1])).mean(dim = 'time_counter').compute()
    zov_periods[time_periods[j] + '-' + time_periods[j+1]] = ds_zov.sel(time_counter = slice(time_periods[j], time_periods[j+1])).mean(dim = 'time_counter').compute()
    ssh_periods_mean[time_periods[j] + '-' + time_periods[j+1]] = ssh_anom.sel(time_counter = slice(time_periods[j], time_periods[j+1])).mean(dim = 'time_counter').compute()
    wind_x_periods[time_periods[j] + '-' + time_periods[j+1]] = ds_wind_x.sel(time_counter = slice(time_periods[j], time_periods[j+1])).mean(dim = 'time_counter').compute()
    wind_y_periods[time_periods[j] + '-' + time_periods[j+1]] = ds_wind_y.sel(time_counter = slice(time_periods[j], time_periods[j+1])).mean(dim = 'time_counter').compute()
    

### SST

In [ ]:
%matplotlib inline
plt.rcParams['font.size'] = 20.0

#Plot Anomalies Temperature
fig = plt.figure(figsize = (30,20)) 

axes = []
for j in range(4):
    ax = fig.add_subplot(2,2, j+1, projection=ccrs.PlateCarree())
    ax.title.set_text(time_periods[j] + '-' + time_periods[j+1])
    atl = WorldMap(central_longitude = -60, real_color = False, zoom = [-85, -40, 30, 57], ax = ax)
    atl.Ax.add_feature(cfeature.LAKES, alpha=0.5)
    atl.Ax.add_feature(cfeature.RIVERS)
    cmesh = atl.Ax.pcolormesh(temp_periods_anom[time_periods[j] + '-' + time_periods[j+1]].nav_lon,
                      temp_periods_anom[time_periods[j] + '-' + time_periods[j+1]].nav_lat,
                      temp_periods_anom[time_periods[j] + '-' + time_periods[j+1]].votemper, 
                      cmap = 'cmo.balance',
                      norm = colors.CenteredNorm())
    atl.Ax.coastlines()
    bthy = atl.add_bathymetry(-1000)
    axes.append(ax)

cbar = fig.colorbar(cmesh, ax = axes, label = '$\u03C3_T$ Anomaly [°C]', location = 'right')

fig.savefig(path_rp + 'DV_NAshelf_NEMO/Plots/Composites/sst_comp.png')
    

### SSS

In [ ]:
%matplotlib inline
plt.rcParams['font.size'] = 20.0

#Plot Anomalies Salinity
fig = plt.figure(figsize = (30,20)) 

axes = []
for j in range(4):
    ax = fig.add_subplot(2,2, j+1, projection=ccrs.PlateCarree())
    ax.title.set_text(time_periods[j] + '-' + time_periods[j+1])
    atl = WorldMap(central_longitude = -60, real_color = False, zoom = [-85, -40, 30, 57], ax = ax)
    atl.Ax.add_feature(cfeature.LAKES, alpha=0.5)
    atl.Ax.add_feature(cfeature.RIVERS)
    cmesh = atl.Ax.pcolormesh(sal_periods_anom[time_periods[j] + '-' + time_periods[j+1]].nav_lon,
                      sal_periods_anom[time_periods[j] + '-' + time_periods[j+1]].nav_lat,
                      sal_periods_anom[time_periods[j] + '-' + time_periods[j+1]].vosaline, 
                      cmap = 'cmo.delta',
                      norm = colors.CenteredNorm())
    atl.Ax.coastlines()
    atl.add_bathymetry(-1000)
    axes.append(ax)

cbar = fig.colorbar(cmesh, ax = axes, label = '$\u03C3_S$ Anomaly [g/kg]', location = 'right')

fig.savefig(path_rp + 'DV_NAshelf_NEMO/Plots/Composites/sss_comp.png')
    

### SSH

In [ ]:
%matplotlib inline
#Plot anomalies
plt.rcParams['font.size'] = 20.0
fig = plt.figure(figsize = (30,20)) 
bmty = xr.open_dataset(path_rp + 'DV_NAshelf_NEMO/Data/bathymetry_NWA.nc')

axes = []
for j in range(4):
    ax = fig.add_subplot(2,2, j+1, projection=ccrs.PlateCarree())
    ax.title.set_text(time_periods[j] + '-' + time_periods[j+1])
    atl = WorldMap(central_longitude = -60, real_color = False, zoom = [-85, -40, 30, 57], ax = ax)
    atl.Ax.add_feature(cfeature.LAKES, alpha=0.5)
    atl.Ax.add_feature(cfeature.RIVERS)
    cmesh = atl.Ax.pcolormesh(ssh_periods_mean['1960-1968'].nav_lon, ssh_periods_mean['1960-1968'].nav_lat,
                      ssh_periods_mean[time_periods[j] + '-' + time_periods[j+1]].sossheig, 
                      cmap = 'cmo.balance',
                      norm = colors.CenteredNorm())
    atl.Ax.coastlines()
    z= - 1000
    cont = atl.Ax.contour(bmty.x, bmty.y, bmty.z, levels = [z], colors = 'black', linewidths = 1)
    atl.Ax.clabel(cont, fontsize = 20)
    axes.append(ax)

cbar = fig.colorbar(cmesh, ax = axes, label = '$\u03C3_{SSH}$ Anomaly [m]', location = 'right')

fig.savefig(path_rp + 'DV_NAshelf_NEMO/Plots/Composites/ssh_comp.png')
    

### Velocity vectors and zonal velocity anomalies

In [ ]:
%matplotlib inline
plt.rcParams['font.size'] = 20.0
#Plot Anomalies Velocities
fig = plt.figure(figsize = (30,20)) 

bmty = xr.open_dataset(path_rp + 'DV_NAshelf_NEMO/Data/bathymetry_NWA.nc')

axes = []
for j in range(4):
    ax = fig.add_subplot(2,2, j+1, projection=ccrs.PlateCarree())
    ax.title.set_text(time_periods[j] + '-' + time_periods[j+1])
    atl = WorldMap(central_longitude = -60, real_color = False, zoom = [-85, -40, 30, 57], ax = ax)
    atl.Ax.add_feature(cfeature.LAKES, alpha=0.5)
    atl.Ax.add_feature(cfeature.RIVERS)
    
    cmesh = atl.Ax.contourf(zov_periods_anom[time_periods[j] + '-' + time_periods[j+1]].nav_lon,
                            zov_periods_anom[time_periods[j] + '-' + time_periods[j+1]].nav_lat,
                            zov_periods_anom[time_periods[j] + '-' + time_periods[j+1]]['vozocrtx'],
                            cmap = 'cmo.balance',
                            levels = 50,
                            norm = colors.CenteredNorm())
    quiv = atl.plot_quiver(zov_periods[time_periods[j] + '-' + time_periods[j+1]].nav_lon,
                           zov_periods[time_periods[j] + '-' + time_periods[j+1]].nav_lat,
                           zov_periods[time_periods[j] + '-' + time_periods[j+1]]['vozocrtx'],
                           mev_periods[time_periods[j] + '-' + time_periods[j+1]]['vomecrty'], 
                           skip = 15)   
    atl.Ax.coastlines()
    
#     atl.Ax.plot(ds_71W.lon, ds_71W.lat, color = 'r', marker = '.', ls = None, label ='71W')
#     atl.Ax.plot(ds_GoM.lon, ds_GoM.lat, color = 'b', marker = '.', ls = None, label ='GoM')
    atl.Ax.plot(ds_OL.lon, ds_OL.lat, color = 'g', marker = '.', ls = None, label ='OL')
    plt.legend()
    
    z = -1000
    cont = atl.Ax.contour(bmty.x, bmty.y, bmty.z, levels = [z], colors = 'black', linewidths = 1)
    atl.Ax.clabel(cont, fontsize = 20)
    #atl.add_bathymetry(-1000)
    axes.append(ax)

cbar = fig.colorbar(cmesh, ax = axes, label = 'Zonal velocity anomaly [m/s]', location = 'right')
    
fig.savefig(path_rp + 'DV_NAshelf_NEMO/Plots/Composites/vel_comp_OL.png')

### Velocity vector anomalies 

In [ ]:
%matplotlib inline
#Plot Anomalies Velocities
fig = plt.figure(figsize = (30,20)) 

axes = []
for j in range(4):
    ax = fig.add_subplot(2,2, j+1, projection=ccrs.PlateCarree())
    ax.title.set_text(time_periods[j] + '-' + time_periods[j+1])
    atl = WorldMap(central_longitude = -60, real_color = False, zoom = [-85, -40, 30, 57], ax = ax)
    atl.Ax.add_feature(cfeature.LAKES, alpha=0.5)
    atl.Ax.add_feature(cfeature.RIVERS)
    quiv = atl.plot_quiver(zov_periods_anom[time_periods[j] + '-' + time_periods[j+1]].nav_lon,
                           zov_periods_anom[time_periods[j] + '-' + time_periods[j+1]].nav_lat,
                           zov_periods_anom[time_periods[j] + '-' + time_periods[j+1]]['vozocrtx'],
                           mev_periods_anom[time_periods[j] + '-' + time_periods[j+1]]['vomecrty'], 
                           skip = 30)
    atl.Ax.coastlines()
    atl.add_bathymetry(-1000)
    axes.append(ax)
    
fig.savefig(path_rp + 'DV_NAshelf_NEMO/Plots/Composites/vel_anom_comp.png')


### Wind Stress

In [ ]:
%matplotlib inline
plt.rcParams['font.size'] = 20.0
#Plot Anomalies Velocities
fig = plt.figure(figsize = (30,20)) 

bmty = xr.open_dataset(path_rp + 'DV_NAshelf_NEMO/Data/bathymetry_NWA.nc')

axes = []
for j in range(4):
    ax = fig.add_subplot(2,2, j+1, projection=ccrs.PlateCarree())
    ax.title.set_text(time_periods[j] + '-' + time_periods[j+1])
    atl = WorldMap(central_longitude = -60, real_color = False, zoom = [-85, -40, 30, 57], ax = ax)
    atl.Ax.add_feature(cfeature.LAKES, alpha=0.5)
    atl.Ax.add_feature(cfeature.RIVERS)
    
    quiv = atl.plot_quiver(wind_x_periods[time_periods[j] + '-' + time_periods[j+1]].nav_lon,
                           wind_x_periods[time_periods[j] + '-' + time_periods[j+1]].nav_lat,
                           wind_x_periods[time_periods[j] + '-' + time_periods[j+1]]['sozotaux'],
                           wind_y_periods[time_periods[j] + '-' + time_periods[j+1]]['sometauy'], 
                           skip = 30)   
    atl.Ax.coastlines()
    
    z = -1000
    cont = atl.Ax.contour(bmty.x, bmty.y, bmty.z, levels = [z], colors = 'black', linewidths = 1)
    atl.Ax.clabel(cont, fontsize = 20)
    atl.add_bathymetry(-1000)
    axes.append(ax)

fig.savefig(path_rp + 'DV_NAshelf_NEMO/Plots/Composites/wind_comp.png')